In [1]:
!pip install deepctr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 16.5 MB/s 
     |████████████████████████████████| 2.9 MB 50.8 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [107]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras_preprocessing.sequence import pad_sequences
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names
from deepctr.models.xdeepfm import xDeepFM
from sklearn.model_selection import train_test_split

In [6]:
# data import
search = pd.read_csv('search.csv', encoding='UTF-8')
detail = pd.read_csv('detail.csv', encoding='UTF-8')
detail_title = pd.read_csv('detail_title.csv', encoding='UTF-8')
wish = pd.read_csv('wish.csv', encoding='UTF-8')
wish_title = pd.read_csv('wish_title.csv', encoding='UTF-8')

In [7]:
# 검색기록 (dataprocessing ; book feauture중, param들어간 book을 해당유저와 join해줘서 user,book feature의 interction labeling)
# ex _ '운동' param가진 user에 해당 키워드를 가진 book을 모두 join후, interaction을 1로 labeling
search

,customer_type,customer_idx,param,created_at
0,user,5147,talk to me in korean,2022-06-28 22:35:11
1,user,5151,허리,2022-06-29 04:02:43
2,user,5151,허리 운동,2022-06-29 04:02:53
3,user,5151,허리 운동,2022-06-29 04:09:23
4,user,5151,허리,2022-06-29 04:09:57
...,...,...,...,...
1148,company_emp,99,근대의 경계에서,2022-09-20 03:03:44
1149,user,5978,집에 있는데도 집에 가고 싶어,2022-09-20 17:10:17
1150,user,5978,82년,2022-09-20 17:10:42
1151,user,5978,82년생 김지영,2022-09-20 17:11:18


In [8]:
detail_title # 상세(?)

,Unnamed: 0,customer_type,customer_idx,isbn,created_at,title,date
0,0,user,53,9781635190151,2022-06-28 10:58:06,Anytime Korean Beginning 1,2020.3.1
1,1,user,5128,9781635190151,2022-06-28 12:20:45,Anytime Korean Beginning 1,2020.3.1
2,2,user,5080,9781635190151,2022-06-28 12:49:22,Anytime Korean Beginning 1,2020.3.1
3,3,user,5109,9781635190151,2022-06-29 09:30:47,Anytime Korean Beginning 1,2020.3.1
4,4,user,5109,9781635190151,2022-06-29 09:52:44,Anytime Korean Beginning 1,2020.3.1
...,...,...,...,...,...,...,...
1711,1711,company_emp,99,9788976829641,2022-09-20 03:03:50,근대의 경계에서 독재를 읽다(반양장),2006.8.30
1712,1712,user,5886,9788973008797,2022-09-20 19:27:11,이화 한국어 1-2(영어판),2010.7.5
1713,1713,user,6009,9791129378583,2022-09-21 01:29:31,어느 날 공주가 되어버렸다 5,2021.7.20
1714,1714,user,6009,9791140401710,2022-09-21 01:29:53,어느 날 공주가 되어버렸다 7,2022.9.27


In [9]:
wish_title # 장바구니

,Unnamed: 0,customer_type,customer_idx,isbn,created_at,title,date
0,0,user,5060,9788974747398,2022-07-01 03:05:10,논어 백가락(가야금 명인 황병기의),2013.10.25
1,1,user,5060,9788928505326,2022-07-01 03:05:18,12현 가야금 배우기 Level 1(가야금은 숙명처럼),2015.5.30
2,2,user,5060,9788956387949,2022-07-01 03:05:30,가야금 교본(스프링),2009.8.31
3,3,user,5060,9788932625065,2022-07-01 03:11:41,가야금교본(증보판)(스프링),2005.6.1
4,4,user,4701,9791138313667,2022-07-05 03:04:54,배워서 바로 써먹는 찰떡 한국어 필수 회화,2022.5.4
5,5,user,663,9781635190380,2022-07-07 02:47:23,Essential Korean Proverbs,2021.12.15
6,6,user,663,9781635190380,2022-07-07 02:48:47,Essential Korean Proverbs,2021.12.15
7,7,user,663,9781635190380,2022-07-07 02:51:49,Essential Korean Proverbs,2021.12.15
8,8,user,663,9781635190380,2022-07-07 02:52:43,Essential Korean Proverbs,2021.12.15
9,9,user,663,9781635190250,2022-07-07 02:50:35,Learning Korean Through Tasks,2021.1.25


In [11]:
## detail_title,wish_title 를 cocat 후 binary task 구현

In [13]:
data = pd.concat([detail_title, wish_title])
data

,Unnamed: 0,customer_type,customer_idx,isbn,created_at,title,date
0,0,user,53,9781635190151,2022-06-28 10:58:06,Anytime Korean Beginning 1,2020.3.1
1,1,user,5128,9781635190151,2022-06-28 12:20:45,Anytime Korean Beginning 1,2020.3.1
2,2,user,5080,9781635190151,2022-06-28 12:49:22,Anytime Korean Beginning 1,2020.3.1
3,3,user,5109,9781635190151,2022-06-29 09:30:47,Anytime Korean Beginning 1,2020.3.1
4,4,user,5109,9781635190151,2022-06-29 09:52:44,Anytime Korean Beginning 1,2020.3.1
...,...,...,...,...,...,...,...
48,48,user,5768,9791196065287,2022-08-29 18:36:41,김 비서가 왜 그럴까 2,2018.5.30
49,49,user,5768,9791130026718,2022-08-29 18:36:44,김 비서가 왜 그럴까 1,2018.2.9
50,50,user,5768,9791130026718,2022-08-29 18:47:17,김 비서가 왜 그럴까 1,2018.2.9
51,51,user,5768,9791130026725,2022-08-29 18:36:46,김 비서가 왜 그럴까 2,2018.2.9


In [21]:
data = data.drop(columns=['Unnamed: 0', 'customer_type','created_at','date'])

In [22]:
data

,customer_idx,isbn,title
0,53,9781635190151,Anytime Korean Beginning 1
1,5128,9781635190151,Anytime Korean Beginning 1
2,5080,9781635190151,Anytime Korean Beginning 1
3,5109,9781635190151,Anytime Korean Beginning 1
4,5109,9781635190151,Anytime Korean Beginning 1
...,...,...,...
48,5768,9791196065287,김 비서가 왜 그럴까 2
49,5768,9791130026718,김 비서가 왜 그럴까 1
50,5768,9791130026718,김 비서가 왜 그럴까 1
51,5768,9791130026725,김 비서가 왜 그럴까 2


In [27]:
def labeling(x):
  return 1

In [32]:
data['customer_idx'] = data['customer_idx'].apply(labeling)
data.rename(columns={'customer_idx' : 'label'},inplace=True)

In [33]:
data # 최종 positive data

,label,isbn,title
0,1,9781635190151,Anytime Korean Beginning 1
1,1,9781635190151,Anytime Korean Beginning 1
2,1,9781635190151,Anytime Korean Beginning 1
3,1,9781635190151,Anytime Korean Beginning 1
4,1,9781635190151,Anytime Korean Beginning 1
...,...,...,...
48,1,9791196065287,김 비서가 왜 그럴까 2
49,1,9791130026718,김 비서가 왜 그럴까 1
50,1,9791130026718,김 비서가 왜 그럴까 1
51,1,9791130026725,김 비서가 왜 그럴까 2


In [19]:
# negative data labeling 생성을 위해 가져옴
data_1 = pd.read_csv('1.csv', sep=';', encoding='cp949')

In [24]:
data_1 = data_1.drop(columns=['introduction', 'publisher_comment','toc'])

In [35]:
data_1['label']=0
data_1

,isbn,title,label
0,2000151036468,성경(46판 2단 금장색인) (BB6 BH) (250130),0
1,2000127000059,남북전육부 율장비교연구(3판),0
2,2000107000024,제7감각을 기르자,0
3,2000156000150,디오니소스의 노동 1,0
4,2000156000198,아침햇살이 그립다,0
...,...,...,...
49996,9788925552774,흔들리지 않고 피는 꽃이 어디 있으랴(양장본 HardCover),0
49997,9788925552750,조용헌의 사주명리학 이야기,0
49998,9788925552798,푸드스타일리스트 할머니가 만든 건 다 맛있어,0
49999,9788925552828,백조도둑,0


In [79]:
df = pd.concat([data, data_1])

In [82]:
df # 최종 데이터

,label,isbn,title
0,1,9781635190151,Anytime Korean Beginning 1
1,1,9781635190151,Anytime Korean Beginning 1
2,1,9781635190151,Anytime Korean Beginning 1
3,1,9781635190151,Anytime Korean Beginning 1
4,1,9781635190151,Anytime Korean Beginning 1
...,...,...,...
49996,0,9788925552774,흔들리지 않고 피는 꽃이 어디 있으랴(양장본 HardCover)
49997,0,9788925552750,조용헌의 사주명리학 이야기
49998,0,9788925552798,푸드스타일리스트 할머니가 만든 건 다 맛있어
49999,0,9788925552828,백조도둑


In [77]:
##################################################################################

In [83]:
sparse_features = ["isbn"]
dense_features = ["title"]

In [84]:
# data inputation for missing values
df[sparse_features] = df[sparse_features].fillna('-1',)
df[dense_features] = df[dense_features].fillna(0, )

In [85]:
df[sparse_features] # 결측값 -1로 채움

,isbn
0,9781635190151
1,9781635190151
2,9781635190151
3,9781635190151
4,9781635190151
...,...
49996,9788925552774
49997,9788925552750
49998,9788925552798
49999,9788925552828


In [86]:
df[dense_features] #결측값 0으로 채움

,title
0,Anytime Korean Beginning 1
1,Anytime Korean Beginning 1
2,Anytime Korean Beginning 1
3,Anytime Korean Beginning 1
4,Anytime Korean Beginning 1
...,...
49996,흔들리지 않고 피는 꽃이 어디 있으랴(양장본 HardCover)
49997,조용헌의 사주명리학 이야기
49998,푸드스타일리스트 할머니가 만든 건 다 맛있어
49999,백조도둑


In [88]:
# creating target variable
target = df['label']

In [91]:
# encoding function  // labelencoder ; 문자를 0부터 증가하는 정수형 숫자로 바꿔주는기능을 제공
def encoding(data,feat,encoder):
    df[feat] = encoder.fit_transform(df[feat])

In [92]:
# encoding for categorical feautures
[encoding(df, feat, LabelEncoder()) for feat in sparse_features]

[None]

In [98]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
  lbe = LabelEncoder()
  df[feat] = lbe.fit_transform(df[feat])

In [93]:
df # 정수 labeling되었음을 확인

,label,isbn,title
0,1,31819,Anytime Korean Beginning 1
1,1,31819,Anytime Korean Beginning 1
2,1,31819,Anytime Korean Beginning 1
3,1,31819,Anytime Korean Beginning 1
4,1,31819,Anytime Korean Beginning 1
...,...,...,...
49996,0,50022,흔들리지 않고 피는 꽃이 어디 있으랴(양장본 HardCover)
49997,0,50021,조용헌의 사주명리학 이야기
49998,0,50023,푸드스타일리스트 할머니가 만든 건 다 맛있어
49999,0,50024,백조도둑


In [101]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in dense_features:
  lbe = LabelEncoder()
  df[feat] = lbe.fit_transform(df[feat])

In [102]:
df

,label,isbn,title
0,1,31819,1226
1,1,31819,1226
2,1,31819,1226
3,1,31819,1226
4,1,31819,1226
...,...,...,...
49996,0,50022,49792
49997,0,50021,39631
49998,0,50023,45777
49999,0,50024,24725


In [104]:
# Using normalization for dense feature
mms = MinMaxScaler(feature_range=(0,1))
df[dense_features] = mms.fit_transform(df[dense_features])
df

,label,isbn,title
0,1,31819,0.024548
1,1,31819,0.024548
2,1,31819,0.024548
3,1,31819,0.024548
4,1,31819,0.024548
...,...,...,...
49996,0,50022,0.996997
49997,0,50021,0.793541
49998,0,50023,0.916603
49999,0,50024,0.495074


In [105]:
# creating a 4 bit embedding for every sparse feature
sparse_feature_columns = [SparseFeat(feat,vocabulary_size = df[feat].nunique(), embedding_dim=4)
                         for i,feat in enumerate(sparse_features)]

In [108]:
# Creating a dense feat
dense_feature_columns = [DenseFeat(feat,1) for feat in dense_features]

In [109]:
dense_feature_columns[0]

DenseFeat(name='title', dimension=1, dtype='float32', transform_fn=None)

In [110]:
# features to be used for dnn part of xdeepfm
dnn_feature_columns = sparse_feature_columns + dense_feature_columns
# features to be used for linear part of xdeepfm
linear_feature_columns = sparse_feature_columns + dense_feature_columns

In [111]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['isbn', 'title']

In [114]:
# creating train test splits
train, test = train_test_split(df, test_size = 0.2) # 200*0.2 = 40개를 test_dataset으로 분리
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}

In [115]:
train

,label,isbn,title
18745,0,18735,0.201954
38300,0,38326,0.738497
34752,0,34775,0.482239
15946,0,15934,0.257859
24596,0,24589,0.478875
...,...,...,...
27108,0,27103,0.473529
32890,0,32912,0.856914
33239,0,33260,0.659385
39013,0,39037,0.670918


In [116]:
train_model_input  # 칼럼명 : 각 칼럼의 row원소

{'isbn': array([18735, 38326, 34775, ..., 33260, 39037,  7667]),
 'title': array([0.20195427, 0.73849666, 0.4822394 , ..., 0.65938489, 0.67091827,
        0.88981218])}

In [117]:
from deepctr.models.xdeepfm import xDeepFM

In [118]:
model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),
cin_layer_size=(128, 128),
cin_split_half=True, cin_activation='relu'
,l2_reg_linear=1e-05,
l2_reg_embedding=1e-05, l2_reg_dnn=0, l2_reg_cin=0,
seed=1024, dnn_dropout=0,dnn_activation='relu',
dnn_use_bn=False, task='binary')

In [119]:
# compiling the model
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

In [122]:
# training the model
history = model.fit(train_model_input, train['label'].values,
                    batch_size = 256, epochs = 30, verbose = 2, validation_split = 0.2, )

Epoch 1/30
130/130 - 10s - loss: 0.1572 - binary_crossentropy: 0.1571 - val_loss: 0.1077 - val_binary_crossentropy: 0.1076
Epoch 2/30
130/130 - 1s - loss: 0.0408 - binary_crossentropy: 0.0406 - val_loss: 0.0803 - val_binary_crossentropy: 0.0798
Epoch 3/30
130/130 - 1s - loss: 0.0021 - binary_crossentropy: 0.0017 - val_loss: 0.0661 - val_binary_crossentropy: 0.0657
Epoch 4/30
130/130 - 1s - loss: 0.0018 - binary_crossentropy: 0.0014 - val_loss: 0.0690 - val_binary_crossentropy: 0.0686
Epoch 5/30
130/130 - 1s - loss: 0.0017 - binary_crossentropy: 0.0013 - val_loss: 0.0907 - val_binary_crossentropy: 0.0903
Epoch 6/30
130/130 - 1s - loss: 0.0015 - binary_crossentropy: 0.0012 - val_loss: 0.0605 - val_binary_crossentropy: 0.0602
Epoch 7/30
130/130 - 1s - loss: 0.0014 - binary_crossentropy: 0.0011 - val_loss: 0.0666 - val_binary_crossentropy: 0.0663
Epoch 8/30
130/130 - 1s - loss: 0.0012 - binary_crossentropy: 8.7542e-04 - val_loss: 0.0672 - val_binary_crossentropy: 0.0668
Epoch 9/30
130/130 

In [123]:
#predicting
pred_ans_xdeep = model.predict(test_model_input, batch_size=256)

In [124]:
for p,t in zip(pred_ans_xdeep, test['label'].values): # 모델의 predict와 실제 label 값
  print(p,t)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[1.1829729e-05] 0
[1.24055205e-05] 0
[6.779031e-06] 0
[5.4928296e-06] 0
[7.642724e-06] 0
[5.0452577e-06] 0
[5.1217135e-06] 0
[4.687857e-05] 0
[5.505269e-06] 0
[5.4812917e-06] 0
[1.19638435e-05] 0
[1.4418911e-05] 0
[1.676175e-05] 0
[3.3313994e-05] 0
[3.2526477e-05] 0
[1.3234267e-05] 0
[2.8617187e-05] 0
[9.696253e-06] 0
[1.2520905e-05] 0
[5.0645453e-06] 0
[1.9268613e-05] 0
[6.179227e-05] 0
[3.8470116e-05] 0
[6.6237208e-06] 0
[6.680864e-05] 0
[1.4586983e-05] 0
[5.282505e-06] 0
[1.5107062e-05] 0
[1.2957062e-05] 0
[2.3010107e-05] 0
[1.6790165e-05] 0
[2.3575201e-05] 1
[2.2056316e-05] 0
[1.9700337e-05] 0
[8.869045e-06] 0
[1.2365526e-05] 0
[8.8681145e-06] 0
[3.2482647e-05] 0
[1.075829e-05] 0
[9.784406e-06] 0
[7.271061e-06] 0
[6.700923e-06] 0
[5.3309013e-06] 0
[1.4501556e-05] 0
[1.9887877e-05] 0
[1.1392895e-05] 0
[7.271129e-05] 0
[1.6565116e-05] 0
[2.1657788e-05] 0
[1.20082395e-05] 0
[5.264078e-06] 0
[9.571854e-06] 0
[2.2314536e-05] 0
[1.2711191e-05] 0
[5.679